# Data Science Bootcamp
# <center> **Aula 05e -- Pandas**

# Introduction

Oftentimes data will come to us with column names, index names, or other naming conventions that we are not satisfied with. In that case, you'll learn how to use pandas functions to change the names of the offending entries to something better.

You'll also explore how to combine data from multiple DataFrames and/or Series.

# Renaming

The first function we'll introduce here is `rename()`, which lets you change index names and/or column names. For example, to change the `points` column in our dataset to `score`, we would do:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('max_rows', 10)
reviews = pd.read_csv("../Pandas/winemag-data-130k-v2.csv", index_col=0)

In [ ]:
reviews.rename(columns={'points': 'score'})

`rename()` lets you rename index _or_ column values by specifying a `index` or `column` keyword parameter, respectively. It supports a variety of input formats, but usually a Python dictionary is the most convenient. Here is an example using it to rename some elements of the index.

In [ ]:
reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})

You'll probably rename columns very often, but rename index values very rarely.  For that, `set_index()` is usually more convenient.

Both the row index and the column index can have their own `name` attribute. The complimentary `rename_axis()` method may be used to change these names. For example:

In [ ]:
reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')

In [ ]:
del reviews

# Combining

When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series in non-trivial ways. Pandas has three core methods for doing this. In order of increasing complexity, these are `concat()`, `join()`, and `merge()`. Most of what `merge()` can do can also be done more simply with `join()`, so we will omit it and focus on the first two functions here.

The simplest combining method is `concat()`. Given a list of elements, this function will smush those elements together along an axis.

We can use `concat()` to smush them together:

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3])

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7])

df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11])

In [ ]:
frames = [df1, df2, df3]

In [ ]:
result = pd.concat(frames)
result

In [ ]:
result = pd.concat(frames, keys=["df1", "df2", "df3"])
result

In [ ]:
result.loc["df2"]

In [ ]:
df4 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7])

In [ ]:
result = pd.concat([df1, df4], axis=1)
result

In [ ]:
result = pd.concat([df1, df4], axis=1, join="outer")
result

In [ ]:
result = pd.concat([df1, df4], axis=1, join="inner")
result

In [ ]:
result = pd.concat([df1, df4], axis=1).reindex(df1.index)
result

## Concatenating using append

In [ ]:
result = df1.append(df2)
result

In [ ]:
result = df1.append(df4, sort=False)
result

Append may take multiple objects to concatenate.

In [ ]:
result = df1.append([df2, df3])
result

### Ignoring indexes

In [ ]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)
result

In [ ]:
result = df1.append(df4, ignore_index=True, sort=False)
result

## Merge

In [ ]:
result = pd.merge(df1, df4, how="outer")
result

In [ ]:
result = pd.merge(df1, df4, how="inner")
result

In [ ]:
result = pd.merge(df1, df4, how="left")
result

In [ ]:
result = pd.merge(df1, df4, how="right")
result

## Joining on index

The middlemost combiner in terms of complexity is `join()`. `join()` lets you combine different DataFrame objects which have an index in common.

In [ ]:
left = df1.copy()
right = df2.copy()

left.join(right, lsuffix='_1', rsuffix='_2')

In [ ]:
right.join(left, lsuffix='_2', rsuffix='_1')

The `lsuffix` and `rsuffix` parameters are necessary here because the data has the same column names in both British and Canadian datasets. If this wasn't true (because, say, we'd renamed them beforehand) we wouldn't need them.

Another example.

In [ ]:
left = pd.DataFrame({"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"])

right = pd.DataFrame({"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"])

In [ ]:
left.join(right)

In [ ]:
left.join(right, how="outer")

In [ ]:
left.join(right, how="inner")

In [ ]:
left.join(right, how="left")

In [ ]:
left.join(right, how="right")

The data alignment here is on the indexes (row labels). This same behavior can be achieved using merge plus additional arguments instructing it to use the indexes:

In [ ]:
result = pd.merge(left, right, left_index=True, right_index=True, how="outer")
result

In [ ]:
result = pd.merge(left, right, left_index=True, right_index=True, how="inner")
result

### Joining key columns on an index

`join()` takes an optional on argument which may be a column or multiple column names, which specifies that the passed DataFrame is to be aligned on that column in the DataFrame. These two function calls are completely equivalent:

In [ ]:
left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key": ["K0", "K1", "K0", "K1"]
    })

right = pd.DataFrame(
    {
        "C": ["C0", "C1"],
        "D": ["D0", "D1"]
    },
    index=["K0", "K1"])

In [ ]:
result = left.join(right, on="key")
result

To join on multiple keys, the passed DataFrame must have a MultiIndex:

In [ ]:
left = pd.DataFrame(
   {
       "A": ["A0", "A1", "A2", "A3"],
       "B": ["B0", "B1", "B2", "B3"],
       "key1": ["K0", "K0", "K1", "K2"],
       "key2": ["K0", "K1", "K0", "K1"],
   })


index = pd.MultiIndex.from_tuples([("K0", "K0"), ("K1", "K0"), ("K2", "K0"), ("K2", "K1")])


right = pd.DataFrame(
    {
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"]
    },
    index=index)

In [ ]:
result = left.join(right, on=["key1", "key2"])
result

In [ ]:
result = left.join(right, on=["key1", "key2"], how="inner")
result

In [ ]:
result = left.join(right, on=["key1", "key2"], how="outer")
result

In [ ]:
result = left.join(right, on=["key1", "key2"], how="left")
result

In [ ]:
result = left.join(right, on=["key1", "key2"], how="right")
result

---
